In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

# Read the dataset

In [2]:
df = pd.read_excel('Raisin_Dataset.xlsx')
df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


### Check the classes

We have 2 classes.

In [3]:
df["Class"].unique()

array(['Kecimen', 'Besni'], dtype=object)

## Separate two classes

In [4]:
dfs = [x for _, x in df.groupby('Class')]
df0 = dfs[1].reset_index(drop=True)
df1 = dfs[0].drop(columns=["Class"]).reset_index(drop=True)

In [5]:
df0.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


### Assign class  number to each class

In [6]:
df0["Class"] = 0
df1["Class"] = 1

# Prepare train and test sets

80\% in train and 20\% in test

In [7]:
train_df0 = df0.iloc[0: int(len(df0)*0.8)]
train_df1 = df1.iloc[0: int(len(df1)*0.8)]

In [8]:
X_train = pd.concat([train_df0.drop(columns=["Class"]), train_df1.drop(columns=["Class"])]).reset_index(drop=True)
X_train

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251
...,...,...,...,...,...,...,...
715,56244,398.802452,182.844046,0.888703,58530,0.656366,1008.134
716,142239,614.834478,297.735347,0.874928,148078,0.643516,1553.114
717,78632,407.940329,245.821198,0.798050,79715,0.689011,1068.727
718,93430,467.637119,258.947168,0.832693,98337,0.712988,1258.966


In [9]:
y_train = pd.concat([train_df0["Class"], train_df1["Class"]])
y_train

0      0
1      0
2      0
3      0
4      0
      ..
355    1
356    1
357    1
358    1
359    1
Name: Class, Length: 720, dtype: int64

In [10]:
test_df0 = df0.iloc[int(len(df0)*0.8):]
test_df1 = df1.iloc[int(len(df1)*0.8):]

In [11]:
X_test = pd.concat([test_df0.drop(columns=["Class"]), test_df1.drop(columns=["Class"])]).reset_index(drop=True)
y_test = pd.concat([test_df0["Class"], test_df1["Class"]])

# Standardize the dataset

Before preparing the model, the entire dataset need to be standardized. Only train set mean and standard deviation are used to standardize the dataset to avoid potential bias. Each feature/attribute $(X_i)$ is subtracted by its mean $\mu$ and divided by its standard deviation $\sigma$. 

$\mu = \frac{1}{N} * X_i$\
$X_i = (X_i - \mu) / \sigma$

In [12]:
mean_train = X_train.mean()
std_train = X_train.std()

In [13]:
mean_train

Area               88043.401389
MajorAxisLength      432.215772
MinorAxisLength      254.705362
Eccentricity           0.780984
ConvexArea         91495.638889
Extent                 0.698941
Perimeter           1169.716692
dtype: float64

In [14]:
std_train

Area               38921.340363
MajorAxisLength      117.019632
MinorAxisLength       49.308385
Eccentricity           0.093355
ConvexArea         40763.058741
Extent                 0.054117
Perimeter            274.748746
dtype: float64

In [15]:
X_train_standard = (X_train - mean_train)/ std_train
X_train_standard.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
0,-0.013345,0.085714,-0.028681,0.415126,-0.023297,1.103335,0.052132
1,-0.330857,-0.218127,-0.236733,0.223030,-0.311719,-0.273694,-0.174453
2,0.072264,0.085894,0.235720,0.186058,0.054494,-1.133249,0.141432
3,-1.082065,-1.244878,-0.931795,-1.028276,-1.083325,0.012163,-1.184918
4,-0.221868,-0.683860,0.732577,-2.324163,-0.246121,1.733843,-0.351105


In [16]:
X_test_standard = (X_test - mean_train)/ std_train
X_test_standard.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
0,-1.008043,-0.854138,-1.305133,0.413987,-0.972661,-0.694778,-0.879715
1,-1.002134,-1.254607,-0.687215,-1.579456,-0.996408,0.292508,-1.194003
2,-0.600555,-0.577963,-0.525359,-0.025955,-0.588073,-0.796336,-0.633396
3,0.393964,0.243161,0.675094,-0.005231,0.345248,0.517888,0.220261
4,-0.361534,-0.623341,0.174981,-1.078837,-0.396011,1.031061,-0.553115


# Prepare the LDA algorithm

Final prediction for a give instance x: $p(y=1|x,\theta) = sigmoid(\beta_1 - \beta_0)^Tx + (\gamma_1 - \gamma_0)$ 

Steps to follow:

* Calculate prior probabilities
* Calculate mean of each class
* Calculate covariance of each class
* Calculate shared cavariance
* Calculate $\gamma$ and $\beta$ of each class

All of the above steps must be done only for train set. Read bellow cells to understand each step.

### Calculate prior probabilities from the train set

$\pi_c = N_c / N$

$N_c$ = number of instance in class c\
$N$ = total number of instance

In [17]:
N = len(X_train)
N0 = len(train_df0)
N1 = len(train_df1)

# Prior probability
prior0 = N0/N
prior1 = N1/N

# number of features
dim = len(X_train.columns)

## Calculate mean of the standardized dataset

In [18]:
mean0 = X_train_standard.iloc[:N0].mean() # mean of class 0
mean1 = X_train_standard.iloc[N0:].mean() # mean of class 1

In [19]:
mean0

Area              -0.635454
MajorAxisLength   -0.683382
MinorAxisLength   -0.505047
Eccentricity      -0.463720
ConvexArea        -0.634048
Extent             0.177058
Perimeter         -0.674763
dtype: float64

In [20]:
mean1

Area               0.635454
MajorAxisLength    0.683382
MinorAxisLength    0.505047
Eccentricity       0.463720
ConvexArea         0.634048
Extent            -0.177058
Perimeter          0.674763
dtype: float64

### Covariance matrix of class 0

$ covariance_c = \frac{1}{N_c}*\sum_{i=1}^{N_c} (x_i - \mu_c) * (x_i - \mu_c)^T $

Here,
$N_c$ = total number of instance in class c\
$\mu_c$ = mean of class c\
$x_i$ = instance i of class c

In [21]:
cov0 = (X_train_standard.iloc[:N0] - mean0).T @ (X_train_standard.iloc[:N0] - mean0)

cov0 = cov0/N0
cov0

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
Area,0.203771,0.212053,0.256834,0.059906,0.209709,-0.022176,0.239897
MajorAxisLength,0.212053,0.273188,0.193276,0.269869,0.225140,-0.110807,0.277821
MinorAxisLength,0.256834,0.193276,0.442563,-0.248361,0.256524,0.094984,0.270437
Eccentricity,0.059906,0.269869,-0.248361,1.070258,0.071640,-0.319448,0.150196
ConvexArea,0.209709,0.225140,0.256524,0.071640,0.219973,-0.046739,0.255736
Extent,-0.022176,-0.110807,0.094984,-0.319448,-0.046739,0.672236,-0.100491
Perimeter,0.239897,0.277821,0.270437,0.150196,0.255736,-0.100491,0.310083


### Covariance matrix of class 1

In [22]:
cov1 = (X_train_standard.iloc[N0:] - mean1).T @ (X_train_standard.iloc[N0:] - mean1)

cov1 = cov1/N1
cov1

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
Area,0.985847,0.782949,0.907188,0.068332,0.973028,0.195394,0.820604
MajorAxisLength,0.782949,0.790014,0.560035,0.298403,0.796132,-0.075753,0.754447
MinorAxisLength,0.907188,0.560035,1.044514,-0.234264,0.887002,0.345692,0.687978
Eccentricity,0.068332,0.298403,-0.234264,0.496892,0.081951,-0.251855,0.169057
ConvexArea,0.973028,0.796132,0.887002,0.081951,0.973215,0.136135,0.838439
Extent,0.195394,-0.075753,0.345692,-0.251855,0.136135,1.262287,-0.035001
Perimeter,0.820604,0.754447,0.687978,0.169057,0.838439,-0.035001,0.776528


### Shared Covariance matrix

$ covariance = \frac{\sum_{c=1}^{C}(N_c -1) * covariance_c}{\sum(N_c -1)} $\
Here,
C = number of classes

In [23]:
cov_shared = ((N0-1)*cov0 + (N1-1)*cov1) / ((N0-1) + (N1-1))
cov_shared

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
Area,0.594809,0.497501,0.582011,0.064119,0.591369,0.086609,0.530251
MajorAxisLength,0.497501,0.531601,0.376655,0.284136,0.510636,-0.093280,0.516134
MinorAxisLength,0.582011,0.376655,0.743538,-0.241312,0.571763,0.220338,0.479207
Eccentricity,0.064119,0.284136,-0.241312,0.783575,0.076796,-0.285652,0.159627
ConvexArea,0.591369,0.510636,0.571763,0.076796,0.596594,0.044698,0.547087
Extent,0.086609,-0.093280,0.220338,-0.285652,0.044698,0.967261,-0.067746
Perimeter,0.530251,0.516134,0.479207,0.159627,0.547087,-0.067746,0.543306


In [24]:
cov_shared_inv = np.linalg.inv(cov_shared)

### Calculate $\gamma$ for both classes

$\gamma_c = -0.5* \mu_c^T * shared\ covariance ^{-1} * \mu_c + log(\pi_c)$

In [25]:
gamma0 = -0.5* (mean0.T @ cov_shared_inv @ mean0) + np.log(prior0)
gamma0

-1.2666157584168314

In [26]:
gamma1 = -0.5* (mean1.T @ cov_shared_inv @ mean1) + np.log(prior1)
gamma1

-1.2666157584168425

### Calculate $\beta$ for both classes

$\beta_c = shared\ covariance ^{-1} * \mu_c $

In [27]:
beta0 = cov_shared_inv @ mean0
beta0

array([-7.18116047,  1.3474777 ,  0.11627356, -0.360775  ,  9.25372734,
        0.05769609, -4.82094483])

In [28]:
beta1 = cov_shared_inv @ mean1
beta1

array([ 7.18116047, -1.3474777 , -0.11627356,  0.360775  , -9.25372734,
       -0.05769609,  4.82094483])

# Prediction

After computing necessary parameters, each instance x is tested.

$ probability = sigmoid(\beta_1 - \beta_0)^Tx + (\gamma_1 - \gamma_0)$

This equation gives the probability between 0 and 1. Then, if it is greater than 0.5, it will be defined as class 1, and if it is less than 0.5, it will be defined as class 0.

## Sigmoid function 
$sigmoid(x) = \frac {1} {1+exp(-x)} $

In [29]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [30]:
def prediction_equation(x):
    return sigmoid(((beta1 - beta0) @ x) + (gamma1 - gamma0))

In [31]:
def prediction(X):
    y_proba = np.array([])
    y_pred = np.array([], dtype=np.int16)

    for i in range(len(X)):
        proba = prediction_equation(X.loc[i])

        if proba > 0.5:
            pred = 1
        else:
            pred = 0
        y_proba = np.append(y_proba, proba)
        y_pred =np.append(y_pred, pred)
    return y_proba, y_pred

# Evaluation Matrices

The following evaluation matrices are used to evaluate the LDA model:
* accuracy = (TN + TP)/ N
* sensitivity = TP / (TP + FN)
* specificity = TN / (TN + FP)
* precision = TP / (TP + FP)
* f1-score = $2*\frac{precision * sensitivity} {precision + sensitivity}$
* log loss = $- \frac{1}{N} * \sum_{i}^{N} [y_i*ln(p_i) + (1-y_i)*ln(1-p_i)]$

Here,

TP= True Positive\
TN= True Negative\
FP= False Positive\
FN= False Negative\
$y_i$ = ground truth of instance i\
$p_i$ = probability of instance of i

In [32]:
def compute_confusion_matrix(true, pred):
    '''Computes a confusion matrix using numpy.'''

    K = len(np.unique(true)) # Number of classes 
    result = np.zeros((K, K))

    for i in range(len(true)):
        result[true[i]][pred[i]] += 1

    return result

In [33]:
def accuracy(conf_matrix):
    tn, fp, fn, tp = conf_matrix.ravel()
    return (tp+tn)/(tp+tn+fp+fn) 

In [34]:
def sensitivity(conf_matrix):
    tn, fp, fn, tp = conf_matrix.ravel()
    return tp/(tp+fn)   

In [35]:
def specificity(conf_matrix):
    tn, fp, fn, tp = conf_matrix.ravel()
    return tn/(tn+fp) 

In [36]:
def precision(conf_matrix):
    tn, fp, fn, tp = conf_matrix.ravel()
    return tp/(tp+fp) 

In [37]:
def f1_score(conf_matrix):
    tn, fp, fn, tp = conf_matrix.ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1_score = 2 * (precision * recall)/(precision + recall)
    return f1_score

In [38]:
def log_loss(ground_truth, y_proba):
    log_loss = -((ground_truth * np.log(y_proba)) + ((1-ground_truth) * np.log(1-y_proba))).mean()
    return log_loss

## Evaluation matrices for train set

In [39]:
y_train_proba, y_train_pred = prediction(X_train_standard)

conf_matrix_train = compute_confusion_matrix(y_train.to_numpy(), y_train_pred)
conf_matrix_train

array([[313.,  47.],
       [ 51., 309.]])

In [40]:
accuracy(conf_matrix_train)

0.8638888888888889

In [41]:
sensitivity(conf_matrix_train)

0.8583333333333333

In [42]:
specificity(conf_matrix_train)

0.8694444444444445

In [43]:
f1_score(conf_matrix_train)

0.8631284916201116

In [44]:
log_loss(y_train, y_train_proba)

0.32903534847857036

## Evaluation matrices for test set

In [45]:
y_test_proba, y_test_pred = prediction(X_test_standard)

conf_matrix_test = compute_confusion_matrix(y_test.to_numpy(), y_test_pred)
conf_matrix_test

array([[81.,  9.],
       [21., 69.]])

In [46]:
accuracy(conf_matrix_test)

0.8333333333333334

In [47]:
sensitivity(conf_matrix_test)

0.7666666666666667

In [48]:
specificity(conf_matrix_test)

0.9

In [49]:
f1_score(conf_matrix_test)

0.8214285714285715

In [50]:
log_loss(y_test, y_test_proba)

0.4196755531592171